### **1. Importing Libraries**

In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

plt.style.use("seaborn-v0_8")

### **2. Load Data**

This workbook contains three sheets:

- Student_Master
- Lecture_List (ignored for now)
- Attendance_Database (main analysis sheet)

**Setting the File Path**

Using structured project paths via pathlib.

In [156]:
from pathlib import Path

# Get project root (one level above notebooks/)
PROJECT_ROOT = Path().resolve().parent

# Define data directory
DATA_DIR = PROJECT_ROOT / "data"

# Define file path
file_path = DATA_DIR / "Dummy_Data.xlsm"

print("Project Root:", PROJECT_ROOT)
print("File Path:", file_path)

Project Root: C:\Users\DELL\Desktop\Tanmay\End Sem PBL\end-sem-practical-pbl\Analysis
File Path: C:\Users\DELL\Desktop\Tanmay\End Sem PBL\end-sem-practical-pbl\Analysis\data\Dummy_Data.xlsm


**Reading the required sheet**

In [157]:
student_master = pd.read_excel(
    file_path,
    sheet_name="Students_Master",
    engine="openpyxl"
)

attendance_df = pd.read_excel(
    file_path,
    sheet_name="Attendance_Database",
    engine="openpyxl"
)

from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path().resolve().parent
DATA_DIR = PROJECT_ROOT / "data"
file_path = DATA_DIR / "Dummy_Data.xlsm"

attendance_df = pd.read_excel(
    file_path,
    sheet_name="Attendance_Database",
    engine="openpyxl"
)

### 3. **Data Inspection & Validation**

Initial structural checks before cleaning and analysis.

In [158]:
attendance_df.head()

,Date,Time,Batch,PRN,Name,Present
0,2026-02-23,09:30:00,D2,2501132081,Deshpande Aditya Milind,1
1,2026-02-23,09:30:00,D2,2501132082,Devisree Chennu,0
2,2026-02-23,09:30:00,D2,2501132083,Smriti Pramanik,0
3,2026-02-23,09:30:00,D2,2501132086,Shirawale Sanskar Jeevan,1
4,2026-02-23,09:30:00,D2,2501132090,Bhosale Sneha Tushar,0


In [159]:
attendance_df.columns

Index(['Date', 'Time', 'Batch', 'PRN', 'Name', 'Present'], dtype='str')

In [160]:
attendance_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     93 non-null     datetime64[us]
 1   Time     93 non-null     object        
 2   Batch    93 non-null     str           
 3   PRN      93 non-null     int64         
 4   Name     93 non-null     str           
 5   Present  93 non-null     int64         
dtypes: datetime64[us](1), int64(2), object(1), str(2)
memory usage: 4.5+ KB


In [161]:
attendance_df.isna().sum()

Date       0
Time       0
Batch      0
PRN        0
Name       0
Present    0
dtype: int64

In [162]:
attendance_df["Present"].value_counts()

Present
0    59
1    34
Name: count, dtype: int64

### **4. Data Cleaning & Type Standardization**

Converting columns to correct data types and ensuring structural consistency.

In [163]:
# Removing the leading and trailing spaces
attendance_df.columns = attendance_df.columns.str.strip()
attendance_df.columns

Index(['Date', 'Time', 'Batch', 'PRN', 'Name', 'Present'], dtype='str')

In [164]:
# Converting the Date Column

attendance_df['Date'] = pd.to_datetime(
    attendance_df['Date'],
    errors="coerce"
) 

attendance_df['Date'].dtype

dtype('<M8[us]')

*Resolving Time Issue*

In [165]:
print("Dtype:", attendance_df["Time"].dtype)
print("\nFirst 10 values:")
print(attendance_df["Time"].head(10))

print("\nUnique sample:")
print(attendance_df["Time"].unique()[:10])

Dtype: object

First 10 values:
0    09:30:00
1    09:30:00
2    09:30:00
3    09:30:00
4    09:30:00
5    09:30:00
6    09:30:00
7    09:30:00
8    09:30:00
9    09:30:00
Name: Time, dtype: object

Unique sample:
[datetime.time(9, 30) datetime.time(11, 30)]


In [166]:
# Converting the Time Column

print("Before conversion : ",attendance_df["Time"].dtype)

attendance_df["Time"] = pd.to_datetime(
    attendance_df["Time"].astype(str),
    format="%H:%M:%S"
)

print("Head : ",attendance_df["Time"].head())
print("Dtype : ",attendance_df["Time"].dtype)

Before conversion :  object
Head :  0   1900-01-01 09:30:00
1   1900-01-01 09:30:00
2   1900-01-01 09:30:00
3   1900-01-01 09:30:00
4   1900-01-01 09:30:00
Name: Time, dtype: datetime64[us]
Dtype :  datetime64[us]


In [167]:
# Ensuring the attendance is only 0 and 1

attendance_df["Present"] = pd.to_numeric(
    attendance_df['Present'],
    errors="coerce"
)

attendance_df['Present'].dtype

dtype('int64')

In [168]:
# Making the PRN as String cause sometimes excel convert them into scientific number

attendance_df["PRN"] = attendance_df["PRN"].astype(str)
attendance_df["PRN"].head()

0    2501132081
1    2501132082
2    2501132083
3    2501132086
4    2501132090
Name: PRN, dtype: str

*Output after Cleaning and Transforming*

In [ ]:
print("After Cleaning and Transforming Entire Data : ")
print("First 5 columns : \n",attendance_df.head())
print("Data Types : \n",attendance_df.dtypes)
print("Information : \n",attendance_df.info())

After Cleaning and Transforming Entire Data : 
First 5 columns : 
         Date                Time Batch         PRN                      Name  Present
0 2026-02-23 1900-01-01 09:30:00    D2  2501132081   Deshpande Aditya Milind        1
1 2026-02-23 1900-01-01 09:30:00    D2  2501132082           Devisree Chennu        0
2 2026-02-23 1900-01-01 09:30:00    D2  2501132083           Smriti Pramanik        0
3 2026-02-23 1900-01-01 09:30:00    D2  2501132086  Shirawale Sanskar Jeevan        1
4 2026-02-23 1900-01-01 09:30:00    D2  2501132090      Bhosale Sneha Tushar        0
Data Types : 
 Date       datetime64[us]
Time       datetime64[us]
Batch                 str
PRN                   str
Name                  str
Present             int64
dtype: object
<class 'pandas.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     93 non-null     datetime64[us]
 1   T